### Avinash Singh 2011MC04

#### Import Libraries

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l1

In [6]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

#### Loading Dataset

In [7]:
df=pd.read_csv('diabetes.csv')

In [8]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


#### Preprocessing

In [9]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [10]:
" Conclusion: Dataset is Free From NaN Values "

' Conclusion: Dataset is Free From NaN Values '

#### Splitting the Input Features and Target Features

In [11]:
X = df.drop('Outcome', 1)
y = df[['Outcome']]

#### One Hot Encoding of the Target Feature

In [12]:
columnTransformer = ColumnTransformer([('encoder',OneHotEncoder(),[0])],remainder='passthrough')

In [13]:
y = np.array(columnTransformer.fit_transform(y), dtype = np.str) 

In [14]:
X.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32


In [15]:
y.shape

(768, 2)

#### Train-Test Split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

#### Model Building kernel_regularizer, bias_regularizer and activity_regularizer in both Hidden layers and set to l1.

In [17]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD

# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01):
    # create model
    model = Sequential()
    model.add(Dense(500, input_dim=8, activation='sigmoid',kernel_regularizer=l1(learn_rate),bias_regularizer=l1(learn_rate),activity_regularizer=l1(learn_rate)))
    model.add(Dense(100, activation='sigmoid', kernel_regularizer=l1(learn_rate),bias_regularizer=l1(learn_rate),activity_regularizer=l1(learn_rate)))
    model.add(Dense(2, activation='softmax'))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=1000, batch_size=70, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train,y_train,validation_data=(X_test, y_test))

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.669361 using {'learn_rate': 0.001}
0.669361 (0.010667) with: {'learn_rate': 0.001}
0.656353 (0.009964) with: {'learn_rate': 0.01}
0.656353 (0.009964) with: {'learn_rate': 0.1}
0.656353 (0.009964) with: {'learn_rate': 0.2}
0.656353 (0.009964) with: {'learn_rate': 0.3}
